**Demo code for Exploratory Report** <br>
https://datastudio.google.com/reporting/ce9b69b3-3d9b-4aee-bb62-7baab90a0eca

In [4]:
#!pip install web3

In [78]:
import os
import logging
from chainlink_pricefeeds import (abi, get_event_abi, get_logs) # see chainlink_pricefeeds.py
import pandas as pd
from web3 import Web3
from eth_utils import (encode_hex, event_abi_to_log_topic)

######### Enable logging for debug
logging.basicConfig(
        format='%(asctime)s %(funcName)s %(levelname)s %(message)s',
        level='INFO',
        datefmt='%Y-%m-%d %H:%M:%S',
        )


# The dafault RPC from ethersjs, change it if it doesn't work: https://infura.io/docs
eth_endpoint = 'https://mainnet.infura.io/v3/1636ab5444b643bf80a2aa188951f755'
op_endpoint = 'https://optimism-mainnet.infura.io/v3/1636ab5444b643bf80a2aa188951f755'#'https://optimism-mainnet.infura.io/v3/ba54284cc2884c3b8019229d5982ad50'
arb_endpoint = 'https://arbitrum-mainnet.infura.io/v3/1636ab5444b643bf80a2aa188951f755'

w3_connection = {'ethereum' : Web3(Web3.HTTPProvider(eth_endpoint, request_kwargs={'timeout': 30})),
                 'optimism' : Web3(Web3.HTTPProvider(op_endpoint, request_kwargs={'timeout': 30})),
                 'arbitrum' : Web3(Web3.HTTPProvider(arb_endpoint, request_kwargs={'timeout': 30})),
                }
logging.info (f'Ethereum connected: {w3_connection["arbitrum"].isConnected()}')


2024-02-21 01:39:53 <module> INFO Ethereum connected: True


In [76]:
op_pricefeed_contracts = {
    
#    'eth-usd' : ['optimism', Web3.toChecksumAddress('0x02f5E9e9dcc66ba6392f6904D5Fcf8625d9B19C9'), 0.015, 'usd', 1200]
    
#    'op-usd' : ['optimism', Web3.toChecksumAddress('0x4F6dFDFd4d68F68b2692E79f9e94796fC8015770'), 0.02, 'usd', 1200]
    
}

eth_pricefeed_contracts = {

    }

arb_pricefeed_contracts = {
    'wbtc-usd' : ['arbitrum', Web3.toChecksumAddress('0xd0C7101eACbB49F3deCcCc166d238410D6D46d57'), 0.0005, 'usd', 86400],
    'eth-usd' : ['arbitrum', Web3.toChecksumAddress('0x639Fe6ab55C921f74e7fac1ee960C0B6293ba612'), 0.0005, 'usd', 86400],
    'arb-usd' : ['arbitrum', Web3.toChecksumAddress('0xb2A824043730FE05F3DA2efaFa1CBbe83fa548D6'), 0.05, 'usd', 86400],
    'link-usd' : ['arbitrum', Web3.toChecksumAddress('0x86E53CF1B870786351Da77A57575e79CB55812CB'), 0.0005, 'usd', 86400],
    'pendle-usd' : ['arbitrum', Web3.toChecksumAddress('0x66853E19d73c0F9301fe099c324A1E9726953433'), 0.0005, 'usd', 86400],
    
    
}

config = {
            'optimism':op_pricefeed_contracts,
            'ethereum':eth_pricefeed_contracts,
            'arbitrum':arb_pricefeed_contracts
         }

In [77]:
from_block = 0

logging.info(f'from_block={from_block}')

df_columns = ['ticker', 'updated_at', 'chain','price', 'price_decimal', 'type', 'threshold', 'base', 'heartbeat', 'decimals', 'blockNumber','tx_hash']
df = pd.DataFrame(columns = df_columns)
    
event_AnswerUpdated = get_event_abi(abi, 'AnswerUpdated')
topic_AnswerUpdated  = encode_hex(event_abi_to_log_topic(event_AnswerUpdated))

for chain in config.keys():
    chain_contracts = config[chain]
    for ticker in chain_contracts.keys():
        logging.info(f'inside ticker: {ticker}')
        w3 = w3_connection[chain_contracts[ticker][0]]
        address = chain_contracts[ticker][1]
        threshold = chain_contracts[ticker][2]
        base = chain_contracts[ticker][3]
        heartbeat = chain_contracts[ticker][4]
        
        decimals = w3.eth.contract(address=address, abi=abi).functions.decimals().call()

        df_ticker = get_logs(w3, address, [topic_AnswerUpdated], [event_AnswerUpdated], from_block)
        if len(df_ticker)>0:
            df_ticker[['chain', 'ticker', 'threshold', 'base', 'heartbeat', 'decimals', 'type']] = [chain , ticker, threshold, base, heartbeat, decimals, 'direct']

            df_ticker['tx_hash']       = df_ticker['transactionHash'].apply(lambda x: x.hex())        
            df_ticker['updated_at']    = df_ticker['args'].apply(lambda x: pd.to_datetime(x['updatedAt'], unit='s'))
            df_ticker['price']         = df_ticker['args'].apply(lambda x: x['current'])
            df_ticker['price_decimal'] = df_ticker['price']/10**(decimals)

            df = pd.concat([df, df_ticker[df_columns]], ignore_index=True)
            logging.info(f'''{ticker.upper()}: observation count = {len(df_ticker)}, price range: {df_ticker['price_decimal'].min()}-{df_ticker['price_decimal'].max()}''')
      
df = df.sort_values(by = 'updated_at').reset_index(drop=True)
pd.set_option("display.precision", 18)

2024-02-20 18:50:09 <module> INFO from_block=0
2024-02-20 18:50:09 <module> INFO inside ticker: wbtc-usd
2024-02-20 18:50:10 <module> INFO inside ticker: eth-usd
2024-02-20 18:50:11 <module> INFO inside ticker: arb-usd
2024-02-20 18:50:11 <module> INFO inside ticker: link-usd
2024-02-20 18:50:12 <module> INFO inside ticker: pendle-usd


In [71]:
display(df)

,ticker,updated_at,chain,price,price_decimal,type,threshold,base,heartbeat,decimals,blockNumber,tx_hash


In [72]:
df[df['chain']=='arbitrum']

,ticker,updated_at,chain,price,price_decimal,type,threshold,base,heartbeat,decimals,blockNumber,tx_hash


In [58]:
pf = df[df['chain']=='arbitrum']

,ticker,updated_at,chain,price,price_decimal,type,threshold,base,heartbeat,decimals,blockNumber,tx_hash


# Exploratory report:

In [19]:
pf = pf[pf['ticker']=='eth-usd']

In [20]:
pf['price_diff'] = pf['price_decimal'].diff()


In [30]:
pf['dp'] = pf['price_diff'].div( pf['price_decimal'])*100
pf['mod_dp'] = pf['dp'].abs()

https://datastudio.google.com/reporting/ce9b69b3-3d9b-4aee-bb62-7baab90a0eca

In [32]:
pf.sort_values(by=['dp'])


,ticker,updated_at,chain,price,price_decimal,type,threshold,base,heartbeat,decimals,blockNumber,tx_hash,price_diff,dp/p,dp,mod_dp
280593,eth-usd,2023-12-11 02:13:01,optimism,221707501606,2217.075016060000052676,direct,0.014999999999999999,usd,1200,8,113331002,0x53b99c1b32df7a6b1240113502617616872d90dae3d0...,-53.238383939999948780,-2.401289246162303659,-2.401289246162303659,2.401289246162303659
299152,eth-usd,2024-01-03 12:09:51,optimism,214406277465,2144.062774649999937537,direct,0.014999999999999999,usd,1200,8,114342507,0x4cd45ded90bce8c14cdd03402f78e43368c3904bffb2...,-46.728481439999995928,-2.179436254968235342,-2.179436254968235342,2.179436254968235342
48803,eth-usd,2022-09-21 18:01:39,optimism,133507386938,1335.073869379999905505,direct,0.014999999999999999,usd,1200,8,24299583,0xe4a3a2b406b0cb34bab404c434d364ce0fd7efd69e89...,-28.361130620000039926,-2.124311715663401934,-2.124311715663401934,2.124311715663401934
237282,eth-usd,2023-08-17 21:42:43,optimism,165284830000,1652.848300000000108412,direct,0.014999999999999999,usd,1200,8,108354893,0x330063807d8717da3e96b7c20a5db056736654f73911...,-32.301699999999982538,-1.954305183361351395,-1.954305183361351395,1.954305183361351395
280596,eth-usd,2023-12-11 02:13:13,optimism,217591624450,2175.916244500000175321,direct,0.014999999999999999,usd,1200,8,113331008,0xb3a2ef371c2ecd36299874aa4c48388380caed2df3f8...,-41.158771559999877354,-1.891560470860755627,-1.891560470860755627,1.891560470860755627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208810,eth-usd,2023-06-06 20:17:37,optimism,188702905320,1887.029053199999907520,direct,0.014999999999999999,usd,1200,8,105241940,0x89e4bb958ba8c1f2ebcaea80bd80b603a8608dd99e70...,41.459053199999971184,2.197054312952640309,2.197054312952640309,2.197054312952640309
1512,eth-usd,2022-06-19 10:16:59,optimism,103146197318,1031.461973180000086359,direct,0.014999999999999999,usd,1200,8,12336207,0x5751bdc96a4764892fb9b544033c25dbb0fb6bc0ec7e...,24.500535140000124557,2.375321221437268093,2.375321221437268093,2.375321221437268093
67086,eth-usd,2022-10-25 17:00:39,optimism,147667975000,1476.679750000000012733,direct,0.014999999999999999,usd,1200,8,31443818,0x149271995924c23b5e6d1937b37f045c14844dc66a36...,42.502793670000073689,2.878267523476235823,2.878267523476235823,2.878267523476235823
122511,eth-usd,2023-01-14 00:41:22,optimism,157420505176,1574.205051760000060312,direct,0.014999999999999999,usd,1200,8,66116229,0xb1e2373bfce54fa49b7fa2c65d6d713ea60baa25771f...,46.999716390000003230,2.985615904195781933,2.985615904195781933,2.985615904195781933
